In [1]:
import matplotlib
matplotlib.use('agg')

In [2]:
%matplotlib inline

In [3]:
import phenom
import os
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# benzoate

In [5]:
ds = phenom.dataset.DataSet.fromDirectory("../data/lund/PA01-benzoate/")
meta = ds.meta
meta["mMAcid"] = meta["mM-acid"]

In [6]:
mnull = phenom.design.Formula(
    ds.meta, "1 + C(mMAcid, Sum, levels=[5, 10, 20, 0]) + C(pH, Sum, levels=[6.5, 6.0, 5.5, 5, 7]) + C(mMAcid, Sum, levels=[5, 10, 20, 0]):C(pH, Sum, levels=[6.5, 6.0, 5.5, 5, 7])")

hierarchy = phenom.design.Formula(ds.meta, "1") + phenom.design.Formula(ds.meta, "C(plate) + 0")
mbatch = mnull * hierarchy

In [7]:
null_phen = phenom.phenotype.Phenotype(ds.data, mnull, model="phenom_deriv.stan")
batch_phen = phenom.phenotype.Phenotype(ds.data, mbatch, model="phenom_deriv.stan",
                                       lengthscale_priors=[[2.5, 0.45]] * 4 + [[100, 20]] * 4, 
                                       alpha_priors= [[10, 10]] * 4 + [[7, 10]] * 4,
                                       minExpectedCross=.01, maxExpectedCross=10,
                                       sigma_prior=[.02, 20])

full_phen = phenom.phenotype.Phenotype(ds.data, 
                                       mbatch, model="phenom_marginal.stan", 
                                       lengthscale_priors=[[2.5, 0.45]] * 4 + [[100, 20]] * 4, 
                                       alpha_priors= [[10, 10]] * 4 + [[7, 10]] * 4,
                                       minExpectedCross=.01, maxExpectedCross=10,
                                       sigma_prior=[.02, 20],
                                       marginal_lengthscale_prior=[4, 1],
                                       marginal_alpha_prior=[2, 20]
                                      )

In [31]:
null_samples = null_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


In [32]:
print(null_samples)

Inference for Stan model: anon_model_7226d8af40a102760f76e1d6f960daef.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]   0.35  1.4e-3   0.05   0.26   0.32   0.34   0.38   0.46   1120    1.0
lengthscale[2]   0.69  6.0e-3    0.2   0.38   0.55   0.66    0.8   1.15   1088    1.0
lengthscale[3]   0.24  8.9e-4   0.03   0.19   0.22   0.24   0.26    0.3    909    1.0
alpha[1]         1.56    0.02   0.51   0.83   1.19   1.47   1.84    2.8   1011    1.0
alpha[2]         0.28  3.7e-3   0.12   0.14    0.2   0.25   0.32   0.59   1028    1.0
alpha[3]         0.29  2.2e-3   0.07    0.2   0.25   0.29   0.33   0.45    910    1.0
sigma             0.5  5.2e-5 4.2e-3   0.49   0.49    0.5    0.5    0.5   6734    1.0
f_eta[1,1]       -1.4    0.01   0.43  -2.36  -1.66  -1.34  -1.08  -0.71    983    1.0
f_eta[2,1]       0.14  3.5e-3   0.14 

In [40]:
null_phen.save("../samples/pseudomonas/benzoate/null")

In [ ]:
batch_samples = batch_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_7226d8af40a102760f76e1d6f960daef NOW.


In [37]:
print(batch_samples)

Inference for Stan model: anon_model_7226d8af40a102760f76e1d6f960daef.
4 chains, each with iter=2000; warmup=1000; thin=1; 
post-warmup draws per chain=1000, total post-warmup draws=4000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
lengthscale[1]    0.7    0.05   0.87   0.28   0.38   0.51   0.73   2.03    302   1.01
lengthscale[2]   0.81    0.02   0.48   0.37   0.54    0.7   0.91   1.96    662    1.0
lengthscale[3]   0.23  1.5e-3   0.03   0.17   0.21   0.23   0.25    0.3    454    1.0
lengthscale[4]   0.27  2.4e-3   0.05   0.19   0.23   0.26   0.29   0.38    401   1.01
lengthscale[5]   0.49    0.02   0.43   0.25   0.34   0.41   0.52   1.09    375    1.0
lengthscale[6]   0.35  2.1e-3   0.05   0.25   0.31   0.34   0.38   0.46    660   1.01
alpha[1]         1.82    0.03   0.83   0.43   1.26   1.67   2.24   3.78    603   1.01
alpha[2]         0.28  4.4e-3   0.13   0.12   0.19   0.24   0.32   0.63    920    1.0
alpha[3]         0.26  3.1e-3   0.07 

In [39]:
batch_phen.save("../samples/pseudomonas/benzoate/batch")

In [ ]:
full_samples = full_phen.samples()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1952fe1216b95ba3e2733fb643947f40 NOW.


In [ ]:
print(full_samples)

In [ ]:
full_phen.save("../samples/pseudomonas/benzoate/full")